In [49]:
import random
import collections
import queue
import argparse
import time

DEFAULT_NUMBER_OF_BUYER = random.randint(3, 10)
DEFAULT_END_TIME = random.randint(100, 200)

DEPARTURE_INTERVAL = random.randint(3, 10)

Event = collections.namedtuple('Event', 'time proc action')
Event(1,"С кем случилось","что случилось")


class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)

    def run(self, end_time):  # <1>
        """Schedule and display events until time is up"""
        # schedule the first event for each cab
        for _, proc in sorted(self.procs.items()):  
            first_event = next(proc)  # <3>
            self.events.put(first_event)  # <4>

        # main loop of the simulation
        sim_time = 0  # <5>
        while sim_time < end_time:
            if self.events.empty(): 
                print('*** end of events ***')
                break

            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event 
            print('buyer:', proc_id, proc_id * '   ', current_event) 
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id] 
            else:
                self.events.put(next_event)
        else: 
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))
# END BUYER_SIMULATOR


def compute_duration(previous_action):
    """Compute action duration using exponential distribution"""
    if previous_action == 'entered the queue':
        # new state is prowling
        interval = 10
    elif previous_action in ['start shop', 'move next']:
        # new state is trip
        interval = 1
    elif previous_action == 'going home':
        interval = 0
    else:
        raise ValueError('Unknown previous_action: %s' % previous_action)
    return interval

class Buyers():
    
    size_queue = 0
    
    # BEGIN BUYER_PROCESS
    
    def buyer_process(self, ident, posit, start_time=0):
        time = yield Event(start_time, ident, 'entered the queue')
        posit = self.size_queue
        self.size_queue += 1
        if (posit <= 3):
            for i in range(posit):
                time = yield Event(time, ident, 'move next')
                posit -= 1
            if (posit == 0):
                time = yield Event(time, ident, 'start shop')
        else:
            yield Event(time, ident, 'going home')

    def main(self, end_time=DEFAULT_END_TIME, num_buyer=DEFAULT_NUMBER_OF_BUYER,
             seed=None):
        """Initialize random generator, build procs and run simulation"""
        if seed is not None:
            random.seed(seed)  # get reproducible results

        buyer = {i: self.buyer_process(i, self.size_queue, i*DEPARTURE_INTERVAL) for i in range(num_buyer)}
        sim = Simulator(buyer)
        sim.run(end_time)



shop_day = Buyers()
shop_day.main()

buyer: 0  Event(time=0, proc=0, action='entered the queue')
buyer: 1     Event(time=5, proc=1, action='entered the queue')
buyer: 0  Event(time=10, proc=0, action='start shop')
buyer: 2        Event(time=10, proc=2, action='entered the queue')
buyer: 1     Event(time=15, proc=1, action='move next')
buyer: 3           Event(time=15, proc=3, action='entered the queue')
buyer: 1     Event(time=16, proc=1, action='start shop')
buyer: 2        Event(time=20, proc=2, action='move next')
buyer: 4              Event(time=20, proc=4, action='entered the queue')
buyer: 2        Event(time=21, proc=2, action='move next')
buyer: 2        Event(time=22, proc=2, action='start shop')
buyer: 3           Event(time=25, proc=3, action='move next')
buyer: 5                 Event(time=25, proc=5, action='entered the queue')
buyer: 3           Event(time=26, proc=3, action='move next')
buyer: 3           Event(time=27, proc=3, action='move next')
buyer: 3           Event(time=28, proc=3, action='start shop